In [46]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 26 23:31:38 2018

@author: alsherman
"""

from TwitterAPI import TwitterAPI

consumer_key = 'xEU8VrsiDidxowYulOBMvfRu1'
consumer_secret = 'jHMPJ87sxxUkO8WJiPcby8582X0ShZzT5vYzbaOaq6CgjCoMer'
access_token_key = '218401227-0hj2RyXwfaA7fIRP3ZjjxtauUeUg3GmQHiKALSGT'
access_token_secret = 'IBmvNwSjwybu5U7L6e8SG6FtYmh79uNAcmCi698lGWe4U'

api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

In [80]:
import json
import boto3
import boto3
import json
from datetime import datetime
import calendar
import random
import time

In [48]:
r = api.request('search/tweets', {'q':'NFLDraft'})
for item in r:
    print(item)
    break

{'created_at': 'Fri Apr 27 04:49:43 +0000 2018', 'id': 989728275642572801, 'id_str': '989728275642572801', 'text': 'RT @NFL: Welcome to the NFL!\n\nClass is in session, rookies. #NFLDraft https://t.co/DKDFFkCHMg', 'truncated': False, 'entities': {'hashtags': [{'text': 'NFLDraft', 'indices': [60, 69]}], 'symbols': [], 'user_mentions': [{'screen_name': 'NFL', 'name': 'NFL', 'id': 19426551, 'id_str': '19426551', 'indices': [3, 7]}], 'urls': [], 'media': [{'id': 989719519718293509, 'id_str': '989719519718293509', 'indices': [70, 93], 'media_url': 'http://pbs.twimg.com/media/DbwwqNAUQAU0evN.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DbwwqNAUQAU0evN.jpg', 'url': 'https://t.co/DKDFFkCHMg', 'display_url': 'pic.twitter.com/DKDFFkCHMg', 'expanded_url': 'https://twitter.com/NFL/status/989719845284425730/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 680, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 1200, 'h': 1200, 'resize': 'fit'}, 

'Fri Apr 27 04:49:43 +0000 2018'

In [16]:
## create a table to store twitter hashtags in DynamoDB
dynamodb = boto3.resource('dynamodb')

table = dynamodb.create_table(
    TableName='hashtags',
    KeySchema=[
        {
            'AttributeName': 'hashtag',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'hashtag',
            'AttributeType': 'S'
        }
    ],
    # pricing determined by ProvisionedThroughput
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)
table.meta.client.get_waiter('table_exists').wait(TableName='hashtags')

In [43]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('hashtags')
client = boto3.client('kinesis')

In [92]:
print(dir(client))

['_PY_TO_OP_NAME', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cache', '_client_config', '_convert_to_request_dict', '_emit_api_params', '_endpoint', '_exceptions', '_exceptions_factory', '_get_waiter_config', '_load_exceptions', '_loader', '_make_api_call', '_register_handlers', '_request_signer', '_response_parser', '_serializer', '_service_model', 'add_tags_to_stream', 'can_paginate', 'create_stream', 'decrease_stream_retention_period', 'delete_stream', 'describe_limits', 'describe_stream', 'describe_stream_summary', 'disable_enhanced_monitoring', 'enable_enhanced_monitoring', 'exceptions', 'generate_presigned_url', 'get_paginator', 'get_records', 'get_shard_iterator', 'g

In [93]:
#response = client.create_stream(
#   StreamName='twitter',
#   ShardCount=1
#)

In [95]:
client.describe_stream(StreamName='twitter')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '549',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 27 Apr 2018 05:26:53 GMT',
   'server': 'Apache-Coyote/1.1',
   'x-amz-id-2': 'YY48TwbuqQGFweZxE8bOZ3hWpXxhN4uKjojajdiayuewS6W6Gu0d+xDCSwX8oNwP6TxR6xWwHsFpUCVIBTeZ2Jkb4yeV8/DH2rjKlz8cjPw=',
   'x-amzn-requestid': 'cd13312e-516d-a576-9e4f-87f367538244'},
  'HTTPStatusCode': 200,
  'RequestId': 'cd13312e-516d-a576-9e4f-87f367538244',
  'RetryAttempts': 0},
 'StreamDescription': {'EncryptionType': 'NONE',
  'EnhancedMonitoring': [{'ShardLevelMetrics': []}],
  'HasMoreShards': False,
  'RetentionPeriodHours': 24,
  'Shards': [{'HashKeyRange': {'EndingHashKey': '340282366920938463463374607431768211455',
     'StartingHashKey': '0'},
    'SequenceNumberRange': {'StartingSequenceNumber': '49583924914938245978286907913744182185606558507432148994'},
    'ShardId': 'shardId-000000000000'}],
  'StreamARN': 'arn:aws:kinesis:us-east-1:923344594760:stream/twitter',
  'Stre

In [100]:
def put_to_stream(thing_id, property_value, property_timestamp):
    payload = {
        'ShardId': 'shardId-000000000000',
        'timestamp': str(property_timestamp),
        'thing_id': thing_id
    }

    print(payload)

    put_response = client.put_record(
        StreamName='twitter',
        Data=json.dumps(payload),
        PartitionKey=thing_id
    )

In [101]:
property_value = random.randint(40, 120)
property_timestamp = calendar.timegm(datetime.utcnow().timetuple())
thing_id = 'aa-bb'
put_to_stream(thing_id, property_value, property_timestamp)

#kinesis = boto3.client('kinesis')
#r = api.request('search/tweets', {'q':'NFLDraft'})
#for ind, item in enumerate(r):
#    put_to_stream(
#          thing_id='filler'
#        , property_timestamp=item['created_at']
#        , property_value=item['text']
#    )
#    break

{'ShardId': 'shardId-000000000000', 'timestamp': '1524807259', 'thing_id': 'aa-bb'}


In [84]:
response = client.describe_stream(StreamName='twitter')
my_shard_id = response['StreamDescription']['Shards'][0]['ShardId']
my_shard_id

'shardId-000000000000'

In [102]:
shard_it = client.get_shard_iterator(
    StreamName='twitter',
    ShardId=my_shard_id,
    ShardIteratorType='LATEST'
)['ShardIterator']

In [103]:
record_response = client.get_records(
    ShardIterator=shard_it,
    Limit=2
)
shard_it = out["NextShardIterator"]
print(out)

{'Records': [], 'NextShardIterator': 'AAAAAAAAAAGNatnXQSbhBOJxSeFmmwehBM8Ew2wuvsngeRcxQ4luMkDYx8bPOtgiz+adsW9p2MFEMX3S5b2L2Ffo7QD/7EGMOqyyCOOGBiBUV6qVpveiOE7tEOTMGZPpXByBRbi5H8+7qL8MBS2RGkQYY/GPgTEZxBb7Ckq6u+4/7EDjyHv8JEHT4yor2DuI1qZmsT6CViAesIkIeshej0nKEIeT+Me1', 'MillisBehindLatest': 0, 'ResponseMetadata': {'RequestId': 'f4f22e85-6ff3-4229-a7ae-9f552d209e8e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'f4f22e85-6ff3-4229-a7ae-9f552d209e8e', 'x-amz-id-2': 'EKwJAyKRFSmmupY5mEB5iAU7ZHhDxnA/y+2jcvrCYl7vRahwIo58e+sjkYyaoLlOvgdVFrO1WOH5RkDkA9bxzYnDmEUduwBQ', 'content-type': 'application/x-amz-json-1.1', 'content-length': '284', 'date': 'Fri, 27 Apr 2018 05:05:20 GMT'}, 'RetryAttempts': 0}}


In [89]:
while 'NextShardIterator' in record_response:
    record_response = client.get_records(
        ShardIterator=record_response['NextShardIterator'],
        Limit=2
    )
    print(record_response)

{'Records': [], 'NextShardIterator': 'AAAAAAAAAAF+pi1I2sL9wHiheyaHVVDu9UbElgQ0dDCxkCueyo3MYojRI+bzb/+7UHfKxM89nkQp18hp+PNY0zbRYZ4vMN9GrLy36R2IX2l4SvMcjVxm32vXCDdPSTgY4XmQfOT6VHglfas2jhkBBH5joeHPvlgmDhfYAFWECEZz/VN4tfYL0h7EiFL6PlPC+GOGAluQZzr4zO/oLQHpAhE+jMTOj1fb', 'MillisBehindLatest': 0, 'ResponseMetadata': {'RequestId': 'f54dac24-838e-7907-a611-1895eea17435', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'f54dac24-838e-7907-a611-1895eea17435', 'x-amz-id-2': 'kSplAs90gqkcc9GnCuy7uaW1+AV1EY9tvWYKCzWFkaHkFcSSUMflLIrEkcNgip70amS3oNrnhZI0pYTZnSFzHsWVeAPlXBZZP6nRr9aT+JU=', 'content-type': 'application/x-amz-json-1.1', 'content-length': '284', 'date': 'Fri, 27 Apr 2018 05:17:37 GMT'}, 'RetryAttempts': 0}}
{'Records': [], 'NextShardIterator': 'AAAAAAAAAAGtU+hqhTrxtTYws78zucfAYShVkHyKPXh6853LVZ2isF5n+IjSEtZSDKkDkKZsQ45g/k/zWc4CUbIrQTs1ZNkgAyX1bjmQ5i3zKQkPBkdWAsrUjm0IXAk8BZPyyaLorlu0xl71LIoP6m8IomxJtjkrDyMhwwwzWzjg7Ckr9zUDCRz+u5nOTorfIUYxO/aYykOM04rU

ProvisionedThroughputExceededException: An error occurred (ProvisionedThroughputExceededException) when calling the GetRecords operation: Rate exceeded for shard shardId-000000000000 in stream twitter under account 923344594760.